# Fastcat Demo: Varian kV OBI

In [60]:
%pylab widget
%load_ext autoreload
%autoreload 2

[2021-06-21 14:57:13,299] {pyplot.py:290} DEBUG - Loaded backend module://ipympl.backend_nbagg version unknown.
Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import fastcat as fc

# Initialize the spectrum

In [62]:
s = fc.calculate_spectrum(100,14,3,50,monitor=None)

In [63]:
s.attenuate(0.4,fc.get_mu(z=13)) # Aluminum inherent filtration
s.attenuate(0.089,fc.get_mu(z=22)) # Titanium beam hardening filter

In [64]:
plt.figure()
s.get_plot(plt.subplot())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Initialize the detector

In [65]:
det = fc.Detector(s,'CZT-342-micrometer')
det.add_focal_spot(0.6)

plt.figure()
det.get_plot(plt.subplot())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Initialize the phantom

In [66]:
phantom = fc.Catphan_404_Devon(hi_res=False)

[2021-06-21 14:58:07,890] {phantoms.py:878} INFO - Phantom is low resolution


# Run the simulation

In [67]:
angles = np.linspace(0,np.pi*2,180,endpoint=False)
phantom.return_projs(det,s,angles,ASG=False)

[2021-06-21 14:58:12,836] {simulate.py:465} INFO - Running Simulations
[2021-06-21 14:58:12,836] {simulate.py:469} INFO -     0.6 mm focal spot added
[2021-06-21 14:58:12,836] {simulate.py:480} INFO -     Simulating 10 keV
[2021-06-21 14:58:15,069] {simulate.py:480} INFO -     Simulating 20 keV
[2021-06-21 14:58:17,287] {simulate.py:480} INFO -     Simulating 30 keV
[2021-06-21 14:58:19,777] {simulate.py:480} INFO -     Simulating 40 keV
[2021-06-21 14:58:22,341] {simulate.py:480} INFO -     Simulating 50 keV
[2021-06-21 14:58:24,817] {simulate.py:480} INFO -     Simulating 60 keV
[2021-06-21 14:58:27,332] {simulate.py:480} INFO -     Simulating 70 keV
[2021-06-21 14:58:29,825] {simulate.py:480} INFO -     Simulating 80 keV
[2021-06-21 14:58:32,438] {simulate.py:480} INFO -     Simulating 90 keV
[2021-06-21 14:58:35,009] {simulate.py:480} INFO -     Simulating 100 keV
[2021-06-21 14:58:37,329] {simulate.py:612} INFO - Weighting simulations
[2021-06-21 14:58:37,330] {simulate.py:686} IN

In [42]:
class IndexTracker(object):
    def __init__(self, ax, X):
        self.ax = ax
        ax.set_title('use scroll wheel to navigate images')

        self.X = X
        rows, cols, self.slices = X.shape
        self.ind = 10 #self.slices//2

        self.im = ax.imshow(self.X[:, :, self.ind])
        self.update()

    def onscroll(self, event):
        print("%s %s" % (event.button, event.step))
        if event.button == 'up':
            self.ind = (self.ind + 1) % self.slices
        else:
            self.ind = (self.ind - 1) % self.slices
        self.update()

    def update(self):
        self.im.set_data(self.X[:, :, self.ind])
        ax.set_ylabel('slice %s' % self.ind)
        self.im.axes.figure.canvas.draw()

fig, ax = plt.subplots(1, 1)

X = np.array(phantom.proj.T)

tracker = IndexTracker(ax, X)

fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
plt.figure()
plt.imshow(phantom.proj[90],cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Reconstruct

In [68]:
phantom.reconstruct('FDK',filt='ram_lak')

plt.figure()
plt.imshow(phantom.img[5],cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …